<a href="https://colab.research.google.com/github/kevinrchilders/computational-number-theory/blob/master/cryptography_chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
# Fast powering algorithm and gcd algorithm

def power(a, b, n):
  a = a%n
  return 1 if b==0 else a**(b%2) * power(a**2, b//2, n) % n

def gcd(a, b):
  return a if b==0 else gcd(b, a%b)

def extended_gcd(a, b):
  u, g, x, y = 1, a, 0, b
  while y != 0:
    q, t = g // y, g % y
    s = u - q*x
    u, g, x, y = x, y, s, t
  v = (g - a*u) // b 
  return g, u, v

def inverse(a, N):
  g, u, v = extended_gcd(a, N)
  if g==1:
    return u % N

# RSA

In [ ]:
# Example of an RSA encrypted message sent from Alice to Bob.

p = 22953686867719691230002707821868552601124472329079            # One of Bob's secret primes
q = 46484729803540183101830167875623788794533441216779            # The other of Bob's secret primes
N = p*q                                                           # Public modulus
print('N =', N)
e = 56239475893201289374058730812734890751                        # Public encryption exponent
print('e =', e)
d = inverse(e, (p-1)*(q-1)//gcd(p-1, q-1))                        # Bob's private decryption exponent

m = 5712389750123478578042937405028357840920938475980712937489570 # Alice's plaintext
c = power(m, e, N)                                                # Alice's encrypted cyphertext
print('Encrypted message to Bob:', c)

plaintext = power(c, d, N)                                        # Bob's decryption of the cyphertext
print('Message successfully decrypted?: ', plaintext == m)

# Carmichael numbers

In [ ]:
# Show that 341 has many witnesses, thus it is composite.

n = 341
print(n)
for a in range(n):
  if power(a, n, n) != a:
    print(a, ' is a witness for ', n)

In [ ]:
# Show that 561 has no witnesses, even though it is composite.

n = 561
for a in range(n):
  if power(a, n, n) != a:
    print(a, ' is a witness for ', n)

print('561/3 =', 561/3)

# Miller-Rabin test

In [ ]:
# Tests a number a as a Miller-Rabin witness for the compositeness of n

def is_mrwitness(a, n):
  # If a and n have a common factor, then n is composite
  if gcd(a, n) != 1:
    return True

  # Write n-1 = 2^k*q with q odd
  k=0
  q=n-1
  while q%2 == 0:
    k += 1
    q = q//2
  
  # If a^q == 1 (mod n) then a is not a Miller-Rabin witness for n
  a = power(a, q, n)
  if a == 1:
    return False
  
  # If a^(2^iq) == -1 (mod n) for some i<k then a is not a Miller-Rabin witness for n
  for i in range(k):
    if a == n-1:
      return False
    a = power(a, 2, n)

  return True # Otherwise a is a Miller-Rabin witness for n

In [ ]:
is_mrwitness(2, 561)

In [ ]:
n = 57132436325427
# n = 571
for i in range(10):
  a = np.random.randint(0, n)
  print(is_mrwitness(a, n))

In [ ]:
# Test for (probable) primality of n using some number of Miller-Rabin trials.

def is_mrprime(n, trials=50):
  for i in range(trials):
    a = int(np.random.rand()*n)
    if is_mrwitness(a, n):
      return False
  return True

In [ ]:
# Show that this works on a 20 digit prime

is_mrprime(54673257461630679457)

In [ ]:
# Show that this works on a 20 digit random number

is_mrprime(15425793095873928757)

In [ ]:
# Generate new 100 digit (probably) prime numbers

N = 2 * 3 * 5 * 7 * 11 * 13 * 17 * 19 * 23 * 29
for K in range(int(10**99/N), int(10**99/N)+100):
  if is_mrprime(N*K + 1):
    print(N*K + 1)

In [ ]:
is_mrprime(999999999999999984332940850056077901690082242304346832414482863079044519106228408984722082029109461, trials=5000)

# Pollard's p-1 factorization algorithm

In [ ]:
# Attempts to factor N using Pollard's p-1 factorization algorithm.  
# Checks for a non-trivial gcd between a^n!-1 and N

def pollard(N, a=2, maxn=1000000):
  for j in np.arange(2, maxn):
    a = power(a, j, N)
    d = gcd(a-1, N)
    if d != 1 and d != N:
      return d
  print('Test failed, try a larger maxn.')
  return None

In [ ]:
# Small example

N = 13927189

p = pollard(N)

p, N/p

In [ ]:
# 10 digit p and q example

N = 19326223710861634601

p = pollard(N, maxn=100000)

p, N/p

In [ ]:
# 20 digit p and q example

N = 48112959837082048697*54673257461630679457

# I factored these primes-1 using the Pollard factorizer.
# 48112959837082048697-1 = 2^3*277*57467*10091*37440323
# 54673257461630679457-1 = 2^5*3*13^2*3225853*1044653923

pollard(N)

In [ ]:
# Factor a random big number (32 digits)

N = 57483025789123127348975895782470

pollard(N)